사용자의 질문 유형을 분류하여 이미지를 생성할건지/웹검색을 할건지? 등 함수 사용에 있어 
대화 진행방향을 설정할 때 사용하면 좋을 것 같다

SemanticRouting

의미 기반으로 사용자의 자연어가 우리가 설정한 분류기준에 어디에 해당하는지 확인하는 작업

In [1]:
from redisvl.extensions.router import Route

Router 설정

In [2]:
technology = Route(
    name = "technology",
    references = [
        "what are the latest advancements in AI?",
        "tell me about the newest gadgets",
        "what's trending in tech?"
    ],
    metadata = {"category" : "tech", "priority" : 1},
    distance_threshold=0.71 # 각 카테고리 별로 유사도 임계치를 개별적으로 설정할 수 있음음
)

sports = Route(
    name = "sports",
    references = [
        "who won the game last night?",
        "tell me about the upcoming sports events",
        "what's the latest in the world of sports?",
        "sports",
        "basketball and football"
    ],
    metadata = {'category' : 'sports', 'priority' : 2},
    distance_threshold=0.72
)

entertainment = Route(
    name = "entertainment",
    references = [
        "what are the top movies right now?",
        "who won the best actor award?",
        "what's new in the entertainment industry?"
    ],
    metadata = {'category' : 'entertainment', 'priority' : 3},
    distance_threshold=0.7
)

SemanticRouter 인스턴스

In [3]:
import os
from redisvl.extensions.router import SemanticRouter
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from redisvl.utils.vectorize import OpenAITextVectorizer

In [5]:
oai = OpenAITextVectorizer(
    model = "text-embedding-3-small",
    api_config = {"api_key" : os.getenv("OPENAI_API_KEY")}
)

10:42:31 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
router = SemanticRouter(
    name = "topic_router",
    vectorizer = oai,
    routes = [technology, sports, entertainment],
    redis_url = "redis://localhost:30000",
    overwrite = True
)

10:42:31 redisvl.index.index INFO   Index already exists, overwriting.
10:42:31 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
10:42:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
10:42:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [7]:
router._index.info()["num_docs"] # reference 개수 확인

11

In [8]:
route_match = router("Can you tell me about the latest in artificial intelligence?")
route_match

10:42:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


RouteMatch(name='technology', distance=0.415602624416)

In [9]:
route_match = router("are aliens real?")
route_match

10:42:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


RouteMatch(name=None, distance=None)

In [10]:
route_matches = router.route_many("How is AI used in basketball?", max_k=3)
route_matches

10:42:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[RouteMatch(name='technology', distance=0.442498147488),
 RouteMatch(name='sports', distance=0.63991138339)]

In [11]:
from redisvl.extensions.router.schema import DistanceAggregationMethod
route_matches = router.route_many("How is AI used in basketball?", aggregation_method=DistanceAggregationMethod.min, max_k=3)
route_matches

10:42:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[RouteMatch(name='technology', distance=0.442498147488),
 RouteMatch(name='sports', distance=0.575279831886)]

In [12]:
route_matches = router.route_many("최근 가장 인기있는 스포츠 스타는 누구야?")
route_matches

10:42:35 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[RouteMatch(name='sports', distance=0.579744656881)]

라우팅 설정 업데이트

In [13]:
from redisvl.extensions.router import RoutingConfig

router.update_routing_config(
    RoutingConfig(aggregation_method=DistanceAggregationMethod.min, max_k=3)
)

In [14]:
route_matches = router.route_many("Lebron James")
route_matches

10:42:35 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[RouteMatch(name='sports', distance=0.673040270805)]

In [15]:
router.to_dict()

{'name': 'topic_router',
 'routes': [{'name': 'technology',
   'references': ['what are the latest advancements in AI?',
    'tell me about the newest gadgets',
    "what's trending in tech?"],
   'metadata': {'category': 'tech', 'priority': 1},
   'distance_threshold': 0.71},
  {'name': 'sports',
   'references': ['who won the game last night?',
    'tell me about the upcoming sports events',
    "what's the latest in the world of sports?",
    'sports',
    'basketball and football'],
   'metadata': {'category': 'sports', 'priority': 2},
   'distance_threshold': 0.72},
  {'name': 'entertainment',
   'references': ['what are the top movies right now?',
    'who won the best actor award?',
    "what's new in the entertainment industry?"],
   'metadata': {'category': 'entertainment', 'priority': 3},
   'distance_threshold': 0.7}],
 'vectorizer': {'type': 'openai', 'model': 'text-embedding-3-small'},
 'routing_config': {'max_k': 3, 'aggregation_method': 'min'}}

In [16]:
router2 = SemanticRouter.from_dict(router.to_dict(), redis_url = "redis://localhost:30000")

assert router2.to_dict() == router.to_dict()

10:42:36 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
10:42:36 redisvl.index.index INFO   Index already exists, not overwriting.


route Referecne 추가

사용자의 질문중 의미있는 질문을 기존의 router reference 에 추가하는 로직을 만들면 라우팅 기능이 점점 좋아질듯

In [17]:
router.add_route_references(route_name="technology", references=["latest AI trends", "new tech gadgets"])

10:42:36 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


['topic_router:technology:f243fb2d073774e81c7815247cb3013794e6225df3cbe3769cee8c6cefaca777',
 'topic_router:technology:7e4bca5853c1c3298b4d001de13c3c7a79a6e0f134f81acc2e7cddbd6845961f']

In [18]:
refs = router.get_route_references(route_name="technology")
refs

[{'id': 'topic_router:technology:149a9c9919c58534aa0f369e85ad95ba7f00aa0513e0f81e2aff2ea4a717b0e0',
  'reference_id': '149a9c9919c58534aa0f369e85ad95ba7f00aa0513e0f81e2aff2ea4a717b0e0',
  'route_name': 'technology',
  'reference': "what's trending in tech?"},
 {'id': 'topic_router:technology:851f51cce5a9ccfbbcb66993908be6b7871479af3e3a4b139ad292a1bf7e0676',
  'reference_id': '851f51cce5a9ccfbbcb66993908be6b7871479af3e3a4b139ad292a1bf7e0676',
  'route_name': 'technology',
  'reference': 'what are the latest advancements in AI?'},
 {'id': 'topic_router:technology:7e4bca5853c1c3298b4d001de13c3c7a79a6e0f134f81acc2e7cddbd6845961f',
  'reference_id': '7e4bca5853c1c3298b4d001de13c3c7a79a6e0f134f81acc2e7cddbd6845961f',
  'route_name': 'technology',
  'reference': 'new tech gadgets'},
 {'id': 'topic_router:technology:85cc73a1437df27caa2f075a29c497e5a2e532023fbb75378aedbae80779ab37',
  'reference_id': '85cc73a1437df27caa2f075a29c497e5a2e532023fbb75378aedbae80779ab37',
  'route_name': 'technology

In [19]:
refs = router.get_route_references(reference_ids=[refs[0]["reference_id"]])
refs

[{'id': 'topic_router:technology:149a9c9919c58534aa0f369e85ad95ba7f00aa0513e0f81e2aff2ea4a717b0e0',
  'reference_id': '149a9c9919c58534aa0f369e85ad95ba7f00aa0513e0f81e2aff2ea4a717b0e0',
  'route_name': 'technology',
  'reference': "what's trending in tech?"}]

In [20]:
deleted_count = router.delete_route_references(route_name="sports")
deleted_count

5

In [21]:
deleted_count = router.delete_route_references(reference_ids=[refs[0]["reference_id"]])
deleted_count

1

In [22]:
# Use clear to flush all routes from the index
router.clear()

In [24]:
refs = router.get_route_references(route_name="technology")
refs

[]

In [27]:
# Use delete to clear the index and remove it completely
router.delete()

RedisSearchError: Error while deleting index: Unknown Index name

In [26]:
refs = router.get_route_references(route_name="technology")
refs

[]